In [10]:
import sqlite3
import os
import sys
import time
import pandas as pd
import numpy as np
import preprocesser
import talib
from datetime import datetime

종목코드열람

In [11]:
# SQLite DB 연결
conn = sqlite3.connect("t8430.db")
# Connection 으로부터 Cursor 생성
cur = conn.cursor()
# SQL 쿼리 실행
query = "select distinct 종목명, 종목코드 \
        from 종목코드 ;"
        #where 종목코드 = '%s'; " % '000020'
cur.execute(query)
# 데이타 Fetch
rows = cur.fetchall()
shcode_list = pd.DataFrame(rows, columns = ['종목명', '종목코드'])
# Connection 닫기
conn.close()

시세데이터 뽑기

In [15]:
def query_ohlcv(shcode):
    # SQLite DB 연결
    conn = sqlite3.connect("t1305_1.db")
    # Connection 으로부터 Cursor 생성
    cur = conn.cursor()
    # SQL 쿼리 실행
    query = "select 일자, 시가, 고가, 저가, 종가, 누적거래량 \
            from 종목별체결조회 \
            where 1=1 \
            and 일자 between '20190101' and '20190927'\
            and 종목코드 = '%s' \
            and 일주월구분 = '%s' ;" % (shcode,'1')
    cur.execute(query)
    # 데이타 Fetch
    query_result = cur.fetchall()
    fin_result = pd.DataFrame(query_result, columns = ['date', 'open', 'high', 'low', 'close', 'volume'])
    # Connection 닫기
    daily = fin_result.sort_values(by = ['date']).reset_index(drop = True)
    conn.close()
    return daily

In [16]:
# array_df: numpy array date, open, high, low, close, volume
def fun_incresing_ratio_test(array_df, limit = 0.15):
    #1) 종가비율 limit 이상
    ratio_list = [y/x-1 for x, y in zip(array_df[:,4][:-1], array_df[:,4][1:])]
    ratio_test = np.array([i >= limit for i in ratio_list])
    ratio_test = np.insert(ratio_test, 0 , 0)
    return ratio_test

In [17]:
def fun_total_volume_test(array_df, min_total_volume = 30000000000):
    
    #2) 해당날 거래대금 300억 이상 거래대금 = 종가 * 거래량
    total_volume_list = array_df[:,4] * array_df[:,5]
    total_volume_test = [i >= min_total_volume  for i in total_volume_list]
    total_volume_test = np.array(total_volume_test[:])
    return total_volume_test


In [18]:
def fun_average_volume_test(array_df, ratio_test, total_volume_test):
    #3) 급등시점 전 평균 20일간 거래량의 1000%
    average_volume_list = ratio_test * total_volume_test
    average_volume_list_index = [ i for i,v in enumerate(average_volume_list) if v == True]
    
    average_volume_test = np.array([0 for i in range(len(average_volume_list))])
    
    for ind in average_volume_list_index:
        if ind < 20:
            average_vol =  array_df[:,5][:ind].sum() / ind
        else:
            average_vol =  array_df[:,5][ind-20:ind].sum() / 20
        if array_df[:,5][ind] >= average_vol * 10:
            average_volume_test[ind] = 1
            
    return average_volume_test

In [42]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.15
volume_money = 5000000000

for i, v in enumerate(shcode_list['종목코드']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['종목명'].iloc[i]))
    
    daily_data = query_ohlcv(v)
    array_daily = np.array(daily_data)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))     
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = np.where(result_test==1)
    final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[i,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['종목명'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '.csv'
result.to_csv(filename, sep=',')

------------------- test 시작_2019-10-6_21:57:22 : , 0 , 000020 , 동화약품 -------------------  
X_1. ratio_test 없음 0 , 000020 , 동화약품
------------------- test 시작_2019-10-6_21:57:22 : , 1 , 000040 , KR모터스 -------------------  
X_1. ratio_test 없음 1 , 000040 , KR모터스
------------------- test 시작_2019-10-6_21:57:22 : , 2 , 000050 , 경방 -------------------  
X_1. ratio_test 없음 2 , 000050 , 경방
------------------- test 시작_2019-10-6_21:57:22 : , 3 , 000060 , 메리츠화재 -------------------  
X_1. ratio_test 없음 3 , 000060 , 메리츠화재
------------------- test 시작_2019-10-6_21:57:23 : , 4 , 000070 , 삼양홀딩스 -------------------  
X_1. ratio_test 없음 4 , 000070 , 삼양홀딩스
------------------- test 시작_2019-10-6_21:57:23 : , 5 , 000075 , 삼양홀딩스우 -------------------  
X_1. ratio_test 없음 5 , 000075 , 삼양홀딩스우
------------------- test 시작_2019-10-6_21:57:23 : , 6 , 000080 , 하이트진로 -------------------  
X_1. ratio_test 없음 6 , 000080 , 하이트진로
------------------- test 시작_2019-10-6_21:57:23 : , 7 , 000087 , 하이트진로2우B -------------------  
X

X_1. ratio_test 없음 53 , 000880 , 한화
------------------- test 시작_2019-10-6_21:57:34 : , 54 , 000885 , 한화우 -------------------  
O_1. ratio_test 통과 54 , 000885 , 한화우
O_2. total_volume_test 통과 54 , 000885 , 한화우
O_3. average_volume_test 통과 54 , 000885 , 한화우
------------------- test 완료 : , 54 , 000885 , 한화우 ------------------- 
------------------- test 시작_2019-10-6_21:57:34 : , 55 , 00088K , 한화3우B -------------------  
X_1. ratio_test 없음 55 , 00088K , 한화3우B
------------------- test 시작_2019-10-6_21:57:34 : , 56 , 000890 , 보해양조 -------------------  
O_1. ratio_test 통과 56 , 000890 , 보해양조
O_2. total_volume_test 통과 56 , 000890 , 보해양조
O_3. average_volume_test 통과 56 , 000890 , 보해양조
------------------- test 완료 : , 56 , 000890 , 보해양조 ------------------- 
------------------- test 시작_2019-10-6_21:57:34 : , 57 , 000910 , 유니온 -------------------  
O_1. ratio_test 통과 57 , 000910 , 유니온
O_2. total_volume_test 통과 57 , 000910 , 유니온
O_3. average_volume_test 통과 57 , 000910 , 유니온
------------------- test 완료 : ,

X_1. ratio_test 없음 102 , 001560 , 제일연마
------------------- test 시작_2019-10-6_21:57:45 : , 103 , 001570 , 금양 -------------------  
X_1. ratio_test 없음 103 , 001570 , 금양
------------------- test 시작_2019-10-6_21:57:45 : , 104 , 001620 , 케이비아이동국실업 -------------------  
X_1. ratio_test 없음 104 , 001620 , 케이비아이동국실업
------------------- test 시작_2019-10-6_21:57:45 : , 105 , 001630 , 종근당홀딩스 -------------------  
X_1. ratio_test 없음 105 , 001630 , 종근당홀딩스
------------------- test 시작_2019-10-6_21:57:45 : , 106 , 001680 , 대상 -------------------  
X_1. ratio_test 없음 106 , 001680 , 대상
------------------- test 시작_2019-10-6_21:57:45 : , 107 , 001685 , 대상우 -------------------  
X_1. ratio_test 없음 107 , 001685 , 대상우
------------------- test 시작_2019-10-6_21:57:46 : , 108 , 001720 , 신영증권 -------------------  
X_1. ratio_test 없음 108 , 001720 , 신영증권
------------------- test 시작_2019-10-6_21:57:46 : , 109 , 001725 , 신영증권우 -------------------  
X_1. ratio_test 없음 109 , 001725 , 신영증권우
------------------- test 시작_201

X_1. ratio_test 없음 148 , 002600 , 조흥
------------------- test 시작_2019-10-6_21:57:55 : , 149 , 002620 , 제일파마홀딩스 -------------------  
X_1. ratio_test 없음 149 , 002620 , 제일파마홀딩스
------------------- test 시작_2019-10-6_21:57:55 : , 150 , 002630 , 오리엔트바이오 -------------------  
X_1. ratio_test 없음 150 , 002630 , 오리엔트바이오
------------------- test 시작_2019-10-6_21:57:55 : , 151 , 002690 , 동일제강 -------------------  
X_1. ratio_test 없음 151 , 002690 , 동일제강
------------------- test 시작_2019-10-6_21:57:55 : , 152 , 002700 , 신일산업 -------------------  
O_1. ratio_test 통과 152 , 002700 , 신일산업
O_2. total_volume_test 통과 152 , 002700 , 신일산업
O_3. average_volume_test 통과 152 , 002700 , 신일산업
------------------- test 완료 : , 152 , 002700 , 신일산업 ------------------- 
------------------- test 시작_2019-10-6_21:57:56 : , 153 , 002710 , TCC스틸 -------------------  
X_1. ratio_test 없음 153 , 002710 , TCC스틸
------------------- test 시작_2019-10-6_21:57:56 : , 154 , 002720 , 국제약품 -------------------  
O_1. ratio_test 통과 154 , 0027

X_1. ratio_test 없음 195 , 003490 , 대한항공
------------------- test 시작_2019-10-6_21:58:5 : , 196 , 003495 , 대한항공우 -------------------  
O_1. ratio_test 통과 196 , 003495 , 대한항공우
O_2. total_volume_test 통과 196 , 003495 , 대한항공우
O_3. average_volume_test 통과 196 , 003495 , 대한항공우
------------------- test 완료 : , 196 , 003495 , 대한항공우 ------------------- 
------------------- test 시작_2019-10-6_21:58:5 : , 197 , 003520 , 영진약품 -------------------  
X_1. ratio_test 없음 197 , 003520 , 영진약품
------------------- test 시작_2019-10-6_21:58:6 : , 198 , 003530 , 한화투자증권 -------------------  
X_1. ratio_test 없음 198 , 003530 , 한화투자증권
------------------- test 시작_2019-10-6_21:58:6 : , 199 , 003535 , 한화투자증권우 -------------------  
O_1. ratio_test 통과 199 , 003535 , 한화투자증권우
O_2. total_volume_test 통과 199 , 003535 , 한화투자증권우
O_3. average_volume_test 통과 199 , 003535 , 한화투자증권우
------------------- test 완료 : , 199 , 003535 , 한화투자증권우 ------------------- 
------------------- test 시작_2019-10-6_21:58:6 : , 200 , 003540 , 대신증권 ---------

...

O_1. ratio_test 통과 2711 , 228760 , 지노믹트리
O_2. total_volume_test 통과 2711 , 228760 , 지노믹트리
X_3. average_volume_test 없음 2711 , 228760 , 지노믹트리
------------------- test 시작_2019-10-6_22:7:20 : , 2712 , 228850 , 레이언스 -------------------  
X_1. ratio_test 없음 2712 , 228850 , 레이언스
------------------- test 시작_2019-10-6_22:7:20 : , 2713 , 230240 , 에치에프알 -------------------  
O_1. ratio_test 통과 2713 , 230240 , 에치에프알
O_2. total_volume_test 통과 2713 , 230240 , 에치에프알
O_3. average_volume_test 통과 2713 , 230240 , 에치에프알
------------------- test 완료 : , 2713 , 230240 , 에치에프알 ------------------- 
------------------- test 시작_2019-10-6_22:7:20 : , 2714 , 230360 , 에코마케팅 -------------------  
O_1. ratio_test 통과 2714 , 230360 , 에코마케팅
O_2. total_volume_test 통과 2714 , 230360 , 에코마케팅
X_3. average_volume_test 없음 2714 , 230360 , 에코마케팅
------------------- test 시작_2019-10-6_22:7:20 : , 2715 , 230980 , 솔트웍스 -------------------  
O_1. ratio_test 통과 2715 , 230980 , 솔트웍스
O_2. total_volume_test 통과 2715 , 230980 , 솔트웍스
O_3. av

O_1. ratio_test 통과 2753 , 251630 , 브이원텍
O_2. total_volume_test 통과 2753 , 251630 , 브이원텍
O_3. average_volume_test 통과 2753 , 251630 , 브이원텍
------------------- test 완료 : , 2753 , 251630 , 브이원텍 ------------------- 
------------------- test 시작_2019-10-6_22:7:29 : , 2754 , 251970 , 펌텍코리아 -------------------  
O_1. ratio_test 통과 2754 , 251970 , 펌텍코리아
O_2. total_volume_test 통과 2754 , 251970 , 펌텍코리아
X_3. average_volume_test 없음 2754 , 251970 , 펌텍코리아
------------------- test 시작_2019-10-6_22:7:29 : , 2755 , 252500 , 세화피앤씨 -------------------  
O_1. ratio_test 통과 2755 , 252500 , 세화피앤씨
O_2. total_volume_test 통과 2755 , 252500 , 세화피앤씨
O_3. average_volume_test 통과 2755 , 252500 , 세화피앤씨
------------------- test 완료 : , 2755 , 252500 , 세화피앤씨 ------------------- 
------------------- test 시작_2019-10-6_22:7:29 : , 2756 , 253450 , 스튜디오드래곤 -------------------  
X_1. ratio_test 없음 2756 , 253450 , 스튜디오드래곤
------------------- test 시작_2019-10-6_22:7:30 : , 2757 , 253590 , 네오셈 -------------------  
X_1. ratio_test 없음

X_1. ratio_test 없음 2793 , 265480 , 미래에셋대우스팩1호
------------------- test 시작_2019-10-6_22:7:38 : , 2794 , 265520 , AP시스템 -------------------  
X_1. ratio_test 없음 2794 , 265520 , AP시스템
------------------- test 시작_2019-10-6_22:7:38 : , 2795 , 265560 , 영화테크 -------------------  
O_1. ratio_test 통과 2795 , 265560 , 영화테크
O_2. total_volume_test 통과 2795 , 265560 , 영화테크
O_3. average_volume_test 통과 2795 , 265560 , 영화테크
------------------- test 완료 : , 2795 , 265560 , 영화테크 ------------------- 
------------------- test 시작_2019-10-6_22:7:38 : , 2796 , 265920 , 한화수성스팩 -------------------  
X_1. ratio_test 없음 2796 , 265920 , 한화수성스팩
------------------- test 시작_2019-10-6_22:7:38 : , 2797 , 267320 , 교보7호스팩 -------------------  
X_1. ratio_test 없음 2797 , 267320 , 교보7호스팩
------------------- test 시작_2019-10-6_22:7:39 : , 2798 , 267790 , 배럴 -------------------  
X_1. ratio_test 없음 2798 , 267790 , 배럴
------------------- test 시작_2019-10-6_22:7:39 : , 2799 , 267980 , 매일유업 -------------------  
X_1. ratio_test 없음 2

O_1. ratio_test 통과 2838 , 298380 , 에이비엘바이오
O_2. total_volume_test 통과 2838 , 298380 , 에이비엘바이오
X_3. average_volume_test 없음 2838 , 298380 , 에이비엘바이오
------------------- test 시작_2019-10-6_22:7:48 : , 2839 , 299170 , IBKS제10호스팩 -------------------  
X_1. ratio_test 없음 2839 , 299170 , IBKS제10호스팩
------------------- test 시작_2019-10-6_22:7:48 : , 2840 , 299660 , 셀리드 -------------------  
O_1. ratio_test 통과 2840 , 299660 , 셀리드
O_2. total_volume_test 통과 2840 , 299660 , 셀리드
X_3. average_volume_test 없음 2840 , 299660 , 셀리드
------------------- test 시작_2019-10-6_22:7:48 : , 2841 , 299900 , 위지윅스튜디오 -------------------  
O_1. ratio_test 통과 2841 , 299900 , 위지윅스튜디오
O_2. total_volume_test 통과 2841 , 299900 , 위지윅스튜디오
O_3. average_volume_test 통과 2841 , 299900 , 위지윅스튜디오
------------------- test 완료 : , 2841 , 299900 , 위지윅스튜디오 ------------------- 
------------------- test 시작_2019-10-6_22:7:48 : , 2842 , 299910 , 베스파 -------------------  
X_1. ratio_test 없음 2842 , 299910 , 베스파
------------------- test 시작_2019-10-

X_1. ratio_test 없음 2883 , 328380 , 미래에셋대우스팩3호
------------------- test 시작_2019-10-6_22:7:58 : , 2884 , 329560 , 상상인이안제2호스팩 -------------------  
X_1. ratio_test 없음 2884 , 329560 , 상상인이안제2호스팩
------------------- test 시작_2019-10-6_22:7:58 : , 2885 , 900040 , 차이나그레이트 -------------------  
O_1. ratio_test 통과 2885 , 900040 , 차이나그레이트
O_2. total_volume_test 통과 2885 , 900040 , 차이나그레이트
O_3. average_volume_test 통과 2885 , 900040 , 차이나그레이트
------------------- test 완료 : , 2885 , 900040 , 차이나그레이트 ------------------- 
------------------- test 시작_2019-10-6_22:7:58 : , 2886 , 900070 , 글로벌에스엠 -------------------  
X_1. ratio_test 없음 2886 , 900070 , 글로벌에스엠
------------------- test 시작_2019-10-6_22:7:58 : , 2887 , 900080 , 에스앤씨엔진그룹 -------------------  
X_1. ratio_test 없음 2887 , 900080 , 에스앤씨엔진그룹
------------------- test 시작_2019-10-6_22:7:58 : , 2888 , 900100 , 뉴프라이드 -------------------  
O_1. ratio_test 통과 2888 , 900100 , 뉴프라이드
O_2. total_volume_test 통과 2888 , 900100 , 뉴프라이드
O_3. average_volume_test 통과 28

In [44]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.15
volume_money = 10000000000

for i, v in enumerate(shcode_list['종목코드']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['종목명'].iloc[i]))
    
    daily_data = query_ohlcv(v)
    array_daily = np.array(daily_data)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))       
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = np.where(result_test==1)
    final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[i,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['종목명'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '.csv'
result.to_csv(filename, sep=',')

------------------- test 시작_2019-10-6_22:9:6 : , 0 , 000020 , 동화약품 -------------------  
X_1. ratio_test 없음 0 , 000020 , 동화약품
------------------- test 시작_2019-10-6_22:9:6 : , 1 , 000040 , KR모터스 -------------------  
X_1. ratio_test 없음 1 , 000040 , KR모터스
------------------- test 시작_2019-10-6_22:9:6 : , 2 , 000050 , 경방 -------------------  
X_1. ratio_test 없음 2 , 000050 , 경방
------------------- test 시작_2019-10-6_22:9:6 : , 3 , 000060 , 메리츠화재 -------------------  
X_1. ratio_test 없음 3 , 000060 , 메리츠화재
------------------- test 시작_2019-10-6_22:9:6 : , 4 , 000070 , 삼양홀딩스 -------------------  
X_1. ratio_test 없음 4 , 000070 , 삼양홀딩스
------------------- test 시작_2019-10-6_22:9:7 : , 5 , 000075 , 삼양홀딩스우 -------------------  
X_1. ratio_test 없음 5 , 000075 , 삼양홀딩스우
------------------- test 시작_2019-10-6_22:9:7 : , 6 , 000080 , 하이트진로 -------------------  
X_1. ratio_test 없음 6 , 000080 , 하이트진로
------------------- test 시작_2019-10-6_22:9:7 : , 7 , 000087 , 하이트진로2우B -------------------  
X_1. ratio_test 없

O_1. ratio_test 통과 54 , 000885 , 한화우
O_2. total_volume_test 통과 54 , 000885 , 한화우
O_3. average_volume_test 통과 54 , 000885 , 한화우
------------------- test 완료 : , 54 , 000885 , 한화우 ------------------- 
------------------- test 시작_2019-10-6_22:9:18 : , 55 , 00088K , 한화3우B -------------------  
X_1. ratio_test 없음 55 , 00088K , 한화3우B
------------------- test 시작_2019-10-6_22:9:18 : , 56 , 000890 , 보해양조 -------------------  
O_1. ratio_test 통과 56 , 000890 , 보해양조
O_2. total_volume_test 통과 56 , 000890 , 보해양조
O_3. average_volume_test 통과 56 , 000890 , 보해양조
------------------- test 완료 : , 56 , 000890 , 보해양조 ------------------- 
------------------- test 시작_2019-10-6_22:9:18 : , 57 , 000910 , 유니온 -------------------  
O_1. ratio_test 통과 57 , 000910 , 유니온
O_2. total_volume_test 통과 57 , 000910 , 유니온
O_3. average_volume_test 통과 57 , 000910 , 유니온
------------------- test 완료 : , 57 , 000910 , 유니온 ------------------- 
------------------- test 시작_2019-10-6_22:9:18 : , 58 , 000950 , 전방 -------------------  
X

X_1. ratio_test 없음 103 , 001570 , 금양
------------------- test 시작_2019-10-6_22:9:28 : , 104 , 001620 , 케이비아이동국실업 -------------------  
X_1. ratio_test 없음 104 , 001620 , 케이비아이동국실업
------------------- test 시작_2019-10-6_22:9:29 : , 105 , 001630 , 종근당홀딩스 -------------------  
X_1. ratio_test 없음 105 , 001630 , 종근당홀딩스
------------------- test 시작_2019-10-6_22:9:29 : , 106 , 001680 , 대상 -------------------  
X_1. ratio_test 없음 106 , 001680 , 대상
------------------- test 시작_2019-10-6_22:9:29 : , 107 , 001685 , 대상우 -------------------  
X_1. ratio_test 없음 107 , 001685 , 대상우
------------------- test 시작_2019-10-6_22:9:29 : , 108 , 001720 , 신영증권 -------------------  
X_1. ratio_test 없음 108 , 001720 , 신영증권
------------------- test 시작_2019-10-6_22:9:29 : , 109 , 001725 , 신영증권우 -------------------  
X_1. ratio_test 없음 109 , 001725 , 신영증권우
------------------- test 시작_2019-10-6_22:9:30 : , 110 , 001740 , SK네트웍스 -------------------  
X_1. ratio_test 없음 110 , 001740 , SK네트웍스
------------------- test 시작_2019

X_1. ratio_test 없음 151 , 002690 , 동일제강
------------------- test 시작_2019-10-6_22:9:39 : , 152 , 002700 , 신일산업 -------------------  
O_1. ratio_test 통과 152 , 002700 , 신일산업
O_2. total_volume_test 통과 152 , 002700 , 신일산업
O_3. average_volume_test 통과 152 , 002700 , 신일산업
------------------- test 완료 : , 152 , 002700 , 신일산업 ------------------- 
------------------- test 시작_2019-10-6_22:9:39 : , 153 , 002710 , TCC스틸 -------------------  
X_1. ratio_test 없음 153 , 002710 , TCC스틸
------------------- test 시작_2019-10-6_22:9:39 : , 154 , 002720 , 국제약품 -------------------  
O_1. ratio_test 통과 154 , 002720 , 국제약품
O_2. total_volume_test 통과 154 , 002720 , 국제약품
O_3. average_volume_test 통과 154 , 002720 , 국제약품
------------------- test 완료 : , 154 , 002720 , 국제약품 ------------------- 
------------------- test 시작_2019-10-6_22:9:40 : , 155 , 002760 , 보락 -------------------  
X_1. ratio_test 없음 155 , 002760 , 보락
------------------- test 시작_2019-10-6_22:9:40 : , 156 , 002780 , 진흥기업 -------------------  
O_1. ratio_te

X_1. ratio_test 없음 198 , 003530 , 한화투자증권
------------------- test 시작_2019-10-6_22:9:49 : , 199 , 003535 , 한화투자증권우 -------------------  
O_1. ratio_test 통과 199 , 003535 , 한화투자증권우
O_2. total_volume_test 통과 199 , 003535 , 한화투자증권우
O_3. average_volume_test 통과 199 , 003535 , 한화투자증권우
------------------- test 완료 : , 199 , 003535 , 한화투자증권우 ------------------- 
------------------- test 시작_2019-10-6_22:9:49 : , 200 , 003540 , 대신증권 -------------------  
X_1. ratio_test 없음 200 , 003540 , 대신증권
------------------- test 시작_2019-10-6_22:9:50 : , 201 , 003545 , 대신증권우 -------------------  
X_1. ratio_test 없음 201 , 003545 , 대신증권우
------------------- test 시작_2019-10-6_22:9:50 : , 202 , 003547 , 대신증권2우B -------------------  
X_1. ratio_test 없음 202 , 003547 , 대신증권2우B
------------------- test 시작_2019-10-6_22:9:50 : , 203 , 003550 , LG -------------------  
X_1. ratio_test 없음 203 , 003550 , LG
------------------- test 시작_2019-10-6_22:9:50 : , 204 , 003555 , LG우 -------------------  
X_1. ratio_test 없음 204 , 00

...

X_1. ratio_test 없음 2699 , 225590 , 패션플랫폼
------------------- test 시작_2019-10-6_22:18:56 : , 2700 , 226340 , 본느 -------------------  
O_1. ratio_test 통과 2700 , 226340 , 본느
O_2. total_volume_test 통과 2700 , 226340 , 본느
O_3. average_volume_test 통과 2700 , 226340 , 본느
------------------- test 완료 : , 2700 , 226340 , 본느 ------------------- 
------------------- test 시작_2019-10-6_22:18:56 : , 2701 , 226350 , 아이엠텍 -------------------  
O_1. ratio_test 통과 2701 , 226350 , 아이엠텍
O_2. total_volume_test 통과 2701 , 226350 , 아이엠텍
X_3. average_volume_test 없음 2701 , 226350 , 아이엠텍
------------------- test 시작_2019-10-6_22:18:56 : , 2702 , 226360 , 이엑스티 -------------------  
O_1. ratio_test 통과 2702 , 226360 , 이엑스티
O_2. total_volume_test 통과 2702 , 226360 , 이엑스티
O_3. average_volume_test 통과 2702 , 226360 , 이엑스티
------------------- test 완료 : , 2702 , 226360 , 이엑스티 ------------------- 
------------------- test 시작_2019-10-6_22:18:56 : , 2703 , 226400 , 오스테오닉 -------------------  
X_1. ratio_test 없음 2703 , 226400 , 오

X_1. ratio_test 없음 2740 , 243070 , 휴온스
------------------- test 시작_2019-10-6_22:19:5 : , 2741 , 243840 , 신흥에스이씨 -------------------  
X_1. ratio_test 없음 2741 , 243840 , 신흥에스이씨
------------------- test 시작_2019-10-6_22:19:5 : , 2742 , 244460 , 올리패스 -------------------  
X_1. ratio_test 없음 2742 , 244460 , 올리패스
------------------- test 시작_2019-10-6_22:19:5 : , 2743 , 245620 , EDGC -------------------  
X_1. ratio_test 없음 2743 , 245620 , EDGC
------------------- test 시작_2019-10-6_22:19:5 : , 2744 , 246690 , TS인베스트먼트 -------------------  
X_1. ratio_test 없음 2744 , 246690 , TS인베스트먼트
------------------- test 시작_2019-10-6_22:19:6 : , 2745 , 246710 , 티앤알바이오팹 -------------------  
O_1. ratio_test 통과 2745 , 246710 , 티앤알바이오팹
O_2. total_volume_test 통과 2745 , 246710 , 티앤알바이오팹
X_3. average_volume_test 없음 2745 , 246710 , 티앤알바이오팹
------------------- test 시작_2019-10-6_22:19:6 : , 2746 , 246720 , 아스타 -------------------  
X_1. ratio_test 없음 2746 , 246720 , 아스타
------------------- test 시작_2019-10-6_22:19:6

X_1. ratio_test 없음 2781 , 263700 , 케어랩스
------------------- test 시작_2019-10-6_22:19:14 : , 2782 , 263720 , 디앤씨미디어 -------------------  
X_1. ratio_test 없음 2782 , 263720 , 디앤씨미디어
------------------- test 시작_2019-10-6_22:19:14 : , 2783 , 263750 , 펄어비스 -------------------  
X_1. ratio_test 없음 2783 , 263750 , 펄어비스
------------------- test 시작_2019-10-6_22:19:14 : , 2784 , 263770 , 유에스티 -------------------  
X_1. ratio_test 없음 2784 , 263770 , 유에스티
------------------- test 시작_2019-10-6_22:19:14 : , 2785 , 263800 , 데이타솔루션 -------------------  
O_1. ratio_test 통과 2785 , 263800 , 데이타솔루션
O_2. total_volume_test 통과 2785 , 263800 , 데이타솔루션
X_3. average_volume_test 없음 2785 , 263800 , 데이타솔루션
------------------- test 시작_2019-10-6_22:19:15 : , 2786 , 263810 , 상신전자 -------------------  
X_1. ratio_test 없음 2786 , 263810 , 상신전자
------------------- test 시작_2019-10-6_22:19:15 : , 2787 , 263860 , 지니언스 -------------------  
X_1. ratio_test 없음 2787 , 263860 , 지니언스
------------------- test 시작_2019-10-6_22:19:15 :

O_1. ratio_test 통과 2826 , 290380 , 대유
O_2. total_volume_test 통과 2826 , 290380 , 대유
O_3. average_volume_test 통과 2826 , 290380 , 대유
------------------- test 완료 : , 2826 , 290380 , 대유 ------------------- 
------------------- test 시작_2019-10-6_22:19:24 : , 2827 , 290550 , 디케이티 -------------------  
O_1. ratio_test 통과 2827 , 290550 , 디케이티
O_2. total_volume_test 통과 2827 , 290550 , 디케이티
O_3. average_volume_test 통과 2827 , 290550 , 디케이티
------------------- test 완료 : , 2827 , 290550 , 디케이티 ------------------- 
------------------- test 시작_2019-10-6_22:19:24 : , 2828 , 290650 , 엘앤씨바이오 -------------------  
X_1. ratio_test 없음 2828 , 290650 , 엘앤씨바이오
------------------- test 시작_2019-10-6_22:19:24 : , 2829 , 290660 , 네오펙트 -------------------  
X_1. ratio_test 없음 2829 , 290660 , 네오펙트
------------------- test 시작_2019-10-6_22:19:24 : , 2830 , 290670 , 대보마그네틱 -------------------  
X_1. ratio_test 없음 2830 , 290670 , 대보마그네틱
------------------- test 시작_2019-10-6_22:19:24 : , 2831 , 290720 , 푸드나무 ------------

O_1. ratio_test 통과 2872 , 317850 , 대모
O_2. total_volume_test 통과 2872 , 317850 , 대모
O_3. average_volume_test 통과 2872 , 317850 , 대모
------------------- test 완료 : , 2872 , 317850 , 대모 ------------------- 
------------------- test 시작_2019-10-6_22:19:34 : , 2873 , 318000 , 한국바이오젠 -------------------  
O_1. ratio_test 통과 2873 , 318000 , 한국바이오젠
O_2. total_volume_test 통과 2873 , 318000 , 한국바이오젠
X_3. average_volume_test 없음 2873 , 318000 , 한국바이오젠
------------------- test 시작_2019-10-6_22:19:34 : , 2874 , 319400 , 엔에이치스팩14호 -------------------  
X_1. ratio_test 없음 2874 , 319400 , 엔에이치스팩14호
------------------- test 시작_2019-10-6_22:19:34 : , 2875 , 319660 , 피에스케이 -------------------  
X_1. ratio_test 없음 2875 , 319660 , 피에스케이
------------------- test 시작_2019-10-6_22:19:34 : , 2876 , 320000 , 하나금융13호스팩 -------------------  
X_1. ratio_test 없음 2876 , 320000 , 하나금융13호스팩
------------------- test 시작_2019-10-6_22:19:35 : , 2877 , 321260 , 유진스팩4호 -------------------  
O_1. ratio_test 통과 2877 , 321260 , 유진스팩4

In [45]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.15
volume_money = 30000000000

for i, v in enumerate(shcode_list['종목코드']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['종목명'].iloc[i]))
    
    daily_data = query_ohlcv(v)
    array_daily = np.array(daily_data)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['종목명'].iloc[i]))     
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = np.where(result_test==1)
    final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[i,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['종목명'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '.csv'
result.to_csv(filename, sep=',')

------------------- test 시작_2019-10-6_22:19:41 : , 0 , 000020 , 동화약품 -------------------  
X_1. ratio_test 없음 0 , 000020 , 동화약품
------------------- test 시작_2019-10-6_22:19:41 : , 1 , 000040 , KR모터스 -------------------  
X_1. ratio_test 없음 1 , 000040 , KR모터스
------------------- test 시작_2019-10-6_22:19:41 : , 2 , 000050 , 경방 -------------------  
X_1. ratio_test 없음 2 , 000050 , 경방
------------------- test 시작_2019-10-6_22:19:41 : , 3 , 000060 , 메리츠화재 -------------------  
X_1. ratio_test 없음 3 , 000060 , 메리츠화재
------------------- test 시작_2019-10-6_22:19:42 : , 4 , 000070 , 삼양홀딩스 -------------------  
X_1. ratio_test 없음 4 , 000070 , 삼양홀딩스
------------------- test 시작_2019-10-6_22:19:42 : , 5 , 000075 , 삼양홀딩스우 -------------------  
X_1. ratio_test 없음 5 , 000075 , 삼양홀딩스우
------------------- test 시작_2019-10-6_22:19:42 : , 6 , 000080 , 하이트진로 -------------------  
X_1. ratio_test 없음 6 , 000080 , 하이트진로
------------------- test 시작_2019-10-6_22:19:42 : , 7 , 000087 , 하이트진로2우B -------------------  
X

O_1. ratio_test 통과 57 , 000910 , 유니온
O_2. total_volume_test 통과 57 , 000910 , 유니온
X_3. average_volume_test 없음 57 , 000910 , 유니온
------------------- test 시작_2019-10-6_22:19:53 : , 58 , 000950 , 전방 -------------------  
X_1. ratio_test 없음 58 , 000950 , 전방
------------------- test 시작_2019-10-6_22:19:54 : , 59 , 000970 , 한국주철관 -------------------  
X_1. ratio_test 없음 59 , 000970 , 한국주철관
------------------- test 시작_2019-10-6_22:19:54 : , 60 , 000990 , DB하이텍 -------------------  
X_1. ratio_test 없음 60 , 000990 , DB하이텍
------------------- test 시작_2019-10-6_22:19:54 : , 61 , 000995 , DB하이텍1우 -------------------  
O_1. ratio_test 통과 61 , 000995 , DB하이텍1우
X_2. total_volume_test 없음 61 , 000995 , DB하이텍1우
------------------- test 시작_2019-10-6_22:19:54 : , 62 , 001020 , 페이퍼코리아 -------------------  
O_1. ratio_test 통과 62 , 001020 , 페이퍼코리아
X_2. total_volume_test 없음 62 , 001020 , 페이퍼코리아
------------------- test 시작_2019-10-6_22:19:54 : , 63 , 001040 , CJ -------------------  
X_1. ratio_test 없음 63 , 0010

X_1. ratio_test 없음 112 , 001750 , 한양증권
------------------- test 시작_2019-10-6_22:20:5 : , 113 , 001755 , 한양증권우 -------------------  
X_1. ratio_test 없음 113 , 001755 , 한양증권우
------------------- test 시작_2019-10-6_22:20:6 : , 114 , 001770 , 신화실업 -------------------  
O_1. ratio_test 통과 114 , 001770 , 신화실업
X_2. total_volume_test 없음 114 , 001770 , 신화실업
------------------- test 시작_2019-10-6_22:20:6 : , 115 , 001780 , 알루코 -------------------  
X_1. ratio_test 없음 115 , 001780 , 알루코
------------------- test 시작_2019-10-6_22:20:6 : , 116 , 001790 , 대한제당 -------------------  
X_1. ratio_test 없음 116 , 001790 , 대한제당
------------------- test 시작_2019-10-6_22:20:6 : , 117 , 001795 , 대한제당우 -------------------  
O_1. ratio_test 통과 117 , 001795 , 대한제당우
X_2. total_volume_test 없음 117 , 001795 , 대한제당우
------------------- test 시작_2019-10-6_22:20:6 : , 118 , 001799 , 대한제당3우B -------------------  
O_1. ratio_test 통과 118 , 001799 , 대한제당3우B
X_2. total_volume_test 없음 118 , 001799 , 대한제당3우B
------------------- test 

O_1. ratio_test 통과 158 , 002787 , 진흥기업2우B
X_2. total_volume_test 없음 158 , 002787 , 진흥기업2우B
------------------- test 시작_2019-10-6_22:20:15 : , 159 , 002790 , 아모레G -------------------  
X_1. ratio_test 없음 159 , 002790 , 아모레G
------------------- test 시작_2019-10-6_22:20:16 : , 160 , 002795 , 아모레G우 -------------------  
X_1. ratio_test 없음 160 , 002795 , 아모레G우
------------------- test 시작_2019-10-6_22:20:16 : , 161 , 002810 , 삼영무역 -------------------  
X_1. ratio_test 없음 161 , 002810 , 삼영무역
------------------- test 시작_2019-10-6_22:20:16 : , 162 , 002820 , 선창산업 -------------------  
X_1. ratio_test 없음 162 , 002820 , 선창산업
------------------- test 시작_2019-10-6_22:20:16 : , 163 , 002840 , 미원상사 -------------------  
O_1. ratio_test 통과 163 , 002840 , 미원상사
X_2. total_volume_test 없음 163 , 002840 , 미원상사
------------------- test 시작_2019-10-6_22:20:16 : , 164 , 002870 , 신풍제지 -------------------  
X_1. ratio_test 없음 164 , 002870 , 신풍제지
------------------- test 시작_2019-10-6_22:20:17 : , 165 , 002880 , 대유에

X_1. ratio_test 없음 211 , 003670 , 포스코케미칼
------------------- test 시작_2019-10-6_22:20:27 : , 212 , 003680 , 한성기업 -------------------  
O_1. ratio_test 통과 212 , 003680 , 한성기업
X_2. total_volume_test 없음 212 , 003680 , 한성기업
------------------- test 시작_2019-10-6_22:20:27 : , 213 , 003690 , 코리안리 -------------------  
X_1. ratio_test 없음 213 , 003690 , 코리안리
------------------- test 시작_2019-10-6_22:20:27 : , 214 , 003720 , 삼영화학 -------------------  
O_1. ratio_test 통과 214 , 003720 , 삼영화학
X_2. total_volume_test 없음 214 , 003720 , 삼영화학
------------------- test 시작_2019-10-6_22:20:28 : , 215 , 003780 , 진양산업 -------------------  
X_1. ratio_test 없음 215 , 003780 , 진양산업
------------------- test 시작_2019-10-6_22:20:28 : , 216 , 003830 , 대한화섬 -------------------  
X_1. ratio_test 없음 216 , 003830 , 대한화섬
------------------- test 시작_2019-10-6_22:20:28 : , 217 , 003850 , 보령제약 -------------------  
O_1. ratio_test 통과 217 , 003850 , 보령제약
O_2. total_volume_test 통과 217 , 003850 , 보령제약
O_3. average_volume_test 통과 2

...

X_1. ratio_test 없음 2691 , 223310 , 이에스브이
------------------- test 시작_2019-10-6_22:29:29 : , 2692 , 224060 , 코디엠 -------------------  
O_1. ratio_test 통과 2692 , 224060 , 코디엠
O_2. total_volume_test 통과 2692 , 224060 , 코디엠
O_3. average_volume_test 통과 2692 , 224060 , 코디엠
------------------- test 완료 : , 2692 , 224060 , 코디엠 ------------------- 
------------------- test 시작_2019-10-6_22:29:29 : , 2693 , 224110 , 에이텍티앤 -------------------  
X_1. ratio_test 없음 2693 , 224110 , 에이텍티앤
------------------- test 시작_2019-10-6_22:29:29 : , 2694 , 225190 , 삼양옵틱스 -------------------  
X_1. ratio_test 없음 2694 , 225190 , 삼양옵틱스
------------------- test 시작_2019-10-6_22:29:29 : , 2695 , 225330 , 씨엠에스에듀 -------------------  
X_1. ratio_test 없음 2695 , 225330 , 씨엠에스에듀
------------------- test 시작_2019-10-6_22:29:29 : , 2696 , 225430 , 케이엠제약 -------------------  
O_1. ratio_test 통과 2696 , 225430 , 케이엠제약
O_2. total_volume_test 통과 2696 , 225430 , 케이엠제약
O_3. average_volume_test 통과 2696 , 225430 , 케이엠제약
----------------

X_1. ratio_test 없음 2734 , 241710 , 코스메카코리아
------------------- test 시작_2019-10-6_22:29:38 : , 2735 , 241770 , 메카로 -------------------  
X_1. ratio_test 없음 2735 , 241770 , 메카로
------------------- test 시작_2019-10-6_22:29:38 : , 2736 , 241790 , 오션브릿지 -------------------  
X_1. ratio_test 없음 2736 , 241790 , 오션브릿지
------------------- test 시작_2019-10-6_22:29:39 : , 2737 , 241820 , 피씨엘 -------------------  
O_1. ratio_test 통과 2737 , 241820 , 피씨엘
X_2. total_volume_test 없음 2737 , 241820 , 피씨엘
------------------- test 시작_2019-10-6_22:29:39 : , 2738 , 241840 , 에이스토리 -------------------  
O_1. ratio_test 통과 2738 , 241840 , 에이스토리
O_2. total_volume_test 통과 2738 , 241840 , 에이스토리
X_3. average_volume_test 없음 2738 , 241840 , 에이스토리
------------------- test 시작_2019-10-6_22:29:39 : , 2739 , 242040 , 나무기술 -------------------  
O_1. ratio_test 통과 2739 , 242040 , 나무기술
O_2. total_volume_test 통과 2739 , 242040 , 나무기술
X_3. average_volume_test 없음 2739 , 242040 , 나무기술
------------------- test 시작_2019-10-6_22:29:39 

O_1. ratio_test 통과 2780 , 263690 , 디알젬
O_2. total_volume_test 통과 2780 , 263690 , 디알젬
O_3. average_volume_test 통과 2780 , 263690 , 디알젬
------------------- test 완료 : , 2780 , 263690 , 디알젬 ------------------- 
------------------- test 시작_2019-10-6_22:29:48 : , 2781 , 263700 , 케어랩스 -------------------  
X_1. ratio_test 없음 2781 , 263700 , 케어랩스
------------------- test 시작_2019-10-6_22:29:48 : , 2782 , 263720 , 디앤씨미디어 -------------------  
X_1. ratio_test 없음 2782 , 263720 , 디앤씨미디어
------------------- test 시작_2019-10-6_22:29:49 : , 2783 , 263750 , 펄어비스 -------------------  
X_1. ratio_test 없음 2783 , 263750 , 펄어비스
------------------- test 시작_2019-10-6_22:29:49 : , 2784 , 263770 , 유에스티 -------------------  
X_1. ratio_test 없음 2784 , 263770 , 유에스티
------------------- test 시작_2019-10-6_22:29:49 : , 2785 , 263800 , 데이타솔루션 -------------------  
O_1. ratio_test 통과 2785 , 263800 , 데이타솔루션
O_2. total_volume_test 통과 2785 , 263800 , 데이타솔루션
X_3. average_volume_test 없음 2785 , 263800 , 데이타솔루션
----------------

O_1. ratio_test 통과 2827 , 290550 , 디케이티
O_2. total_volume_test 통과 2827 , 290550 , 디케이티
X_3. average_volume_test 없음 2827 , 290550 , 디케이티
------------------- test 시작_2019-10-6_22:29:59 : , 2828 , 290650 , 엘앤씨바이오 -------------------  
X_1. ratio_test 없음 2828 , 290650 , 엘앤씨바이오
------------------- test 시작_2019-10-6_22:29:59 : , 2829 , 290660 , 네오펙트 -------------------  
X_1. ratio_test 없음 2829 , 290660 , 네오펙트
------------------- test 시작_2019-10-6_22:29:59 : , 2830 , 290670 , 대보마그네틱 -------------------  
X_1. ratio_test 없음 2830 , 290670 , 대보마그네틱
------------------- test 시작_2019-10-6_22:29:59 : , 2831 , 290720 , 푸드나무 -------------------  
X_1. ratio_test 없음 2831 , 290720 , 푸드나무
------------------- test 시작_2019-10-6_22:29:59 : , 2832 , 290740 , 액트로 -------------------  
X_1. ratio_test 없음 2832 , 290740 , 액트로
------------------- test 시작_2019-10-6_22:30:0 : , 2833 , 291210 , 한국제7호스팩 -------------------  
X_1. ratio_test 없음 2833 , 291210 , 한국제7호스팩
------------------- test 시작_2019-10-6_22:30:0 : ,

X_1. ratio_test 없음 2874 , 319400 , 엔에이치스팩14호
------------------- test 시작_2019-10-6_22:30:9 : , 2875 , 319660 , 피에스케이 -------------------  
X_1. ratio_test 없음 2875 , 319660 , 피에스케이
------------------- test 시작_2019-10-6_22:30:9 : , 2876 , 320000 , 하나금융13호스팩 -------------------  
X_1. ratio_test 없음 2876 , 320000 , 하나금융13호스팩
------------------- test 시작_2019-10-6_22:30:9 : , 2877 , 321260 , 유진스팩4호 -------------------  
O_1. ratio_test 통과 2877 , 321260 , 유진스팩4호
O_2. total_volume_test 통과 2877 , 321260 , 유진스팩4호
X_3. average_volume_test 없음 2877 , 321260 , 유진스팩4호
------------------- test 시작_2019-10-6_22:30:9 : , 2878 , 322780 , DB금융스팩7호 -------------------  
O_1. ratio_test 통과 2878 , 322780 , DB금융스팩7호
O_2. total_volume_test 통과 2878 , 322780 , DB금융스팩7호
X_3. average_volume_test 없음 2878 , 322780 , DB금융스팩7호
------------------- test 시작_2019-10-6_22:30:10 : , 2879 , 323210 , 이베스트이안스팩1호 -------------------  
O_1. ratio_test 통과 2879 , 323210 , 이베스트이안스팩1호
O_2. total_volume_test 통과 2879 , 323210 , 이베스트이안스

In [26]:
type(filename)

str

In [166]:
df = pd.read_csv("test_result.csv", dtype={'shcode': str})

In [ ]:
df.shape

In [ ]:
df.head(20)

In [ ]:
for i,v in enumerate(df['index']):
    